# Problemas 9: 

Este notebook contiene la solución explicada de 

Implementa un método, has_cycle, que comprueba si un grafo 
 dirigido contiene un grafo. 
- Implementa una solución que utilice el algoritmo dfs (búsqueda en profundida). 
- Implementa una solución que utilice el algoritmo bfs (busquedaen amplitud).


La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



In [6]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        return str(self.vertex)

Esta implementación es la de un grafo dirigido (prescindimos del atributo directed) y adaptamos el código en add_edge y remove_edge: 

In [7]:
from queue import Queue

class Graph:
    def __init__(self, vertices: list) -> None:
        """ We use a dictionary to represent the graph
        the dictionary's keys are the vertices
        The value associated for a given key will be the list of their neighbours.
        Initially, the list of neighbours is empty"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))


    def contain_edge(self, start: object, end: object) -> int:
        """ checks if the edge (start, end) exits. It does
        not exist return 0, eoc returns its weight or 1 (for unweighted graphs)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return 0
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return 0

        # we search the AdjacentVertex whose v is equal to end

        for adj in self._vertices[start]:
            if adj.vertex == end:
                return adj.weight

        return 0  # does not exist

    def remove_edge(self, start: object, end: object):
        """ removes the edge (start, end)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # we must look for the adjacent AdjacentVertex (neighbour)  whose vertex is end, and then remove it
        exist = False
        for adj in self._vertices[start]:
            if adj.vertex == end:
                exist = True
                self._vertices[start].remove(adj)
        

        if not exist: 
            print("({},{}) does not exist!!!!".format(start, end))


    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result




Vamos a extender Graph para que contenga los recorridos dfs y bfs:

In [11]:

class Graph2(Graph):
    def has_cycle_bfs(self) -> bool:
        # we use a dictionary that saves the number of incoming edges for each vertex 
        in_degree=dict.fromkeys(self._vertices.keys(), 0)
        # first, we update the dictionary for those vertices that are adjacent one to others. 
        for v in self._vertices.keys():
            for u in self.get_adjacent_vertices(v):
                in_degree[u] += 1 
 
     
        # We save int o queue those vertices with in-degree equal to 0, that is, those vertices that do not have any incoming edge
        queue=[]
        for v in self._vertices.keys():
            if in_degree[v]==0:
                queue.append(v)
        
        vertex_visited = 0
        # while the queue has elements
        while len(queue) > 0:
            # increase the number of visited nodes
            vertex_visited += 1
            # get the first
            u = queue.pop(0)
            # visits its adjacent vertices
            for v in self.get_adjacent_vertices(u):
                in_degree[v] -= 1   
                if in_degree[v] == 0:   # when it is 0, we have to add to queue to visit it
                    queue.append(v)
            
        # Check if there was a cycle
        
        if vertex_visited==len(self._vertices.keys()):
            return False
        else:
            return True


    def has_cycle_dfs(self) -> bool:
        visited = dict.fromkeys(self._vertices.keys(), False)
        # save the vertices in the recursive calls
        path_recursive = dict.fromkeys(self._vertices.keys(), False)

        for v in self._vertices.keys():
            if not visited[v]:
                if self._has_cycle_dfs(v, visited, path_recursive):
                    return True
        return False

    def _has_cycle_dfs(self, v: object, visited: dict, path_recursive: dict) -> bool:
        # Mark current node as visited and
        # adds to recursion stack
        visited[v] = True
        path_recursive[v] = True
 
        
        for u in self.get_adjacent_vertices(v):
            if not visited[u]:
                if self._has_cycle_dfs(u, visited, path_recursive):
                    return True
            elif path_recursive[u] == True:
                # if u was already visited and also it belongs to the path_recursive, this means that there is a cycle
                return True
 
        # After visiting all its adjacent vertices, we must remove it from path_recursion
        path_recursive[v] = False
        return False


<img src='https://static.javatpoint.com/ds/images/breadth-first-search-algorithm-example.png' widht='300'>

In [12]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
g = Graph2(labels)

g.add_edge('A','B')
g.add_edge('A','D')

g.add_edge('B','C')
g.add_edge('B','F')

g.add_edge('C','E')
g.add_edge('C','G')

g.add_edge('D','F')

g.add_edge('E','F')
g.add_edge('E','B')

g.add_edge('F','A')

g.add_edge('G','E')

g.has_cycle_bfs()


True

## Problema:
En el grafo anterior, elimina las aristas necesarias para que no haya ningún ciclo, y comprueba el método. 